In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [0]:
batch_size = 100
trainset = datasets.MNIST(root='./mnist_data/', train=True,
                               transform=transforms.ToTensor(), download=True)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=batch_size,
                                          shuffle=True, drop_last=True)
testset = datasets.MNIST(root='./mnist_data/', train=False,
                               transform=transforms.ToTensor(), download=True)
testloader = torch.utils.data.DataLoader(testset,batch_size=batch_size,
                                          shuffle=True, drop_last=True)

In [0]:
class VAE(nn.Module):
  def __init__(self):
    super(VAE, self).__init__()
    self.fc1 = nn.Linear(784,512)
    self.fc2 = nn.Linear(512,256)
    self.fc31 = nn.Linear(256,2)
    self.fc32 = nn.Linear(256,2)
    self.fc4 = nn.Linear(2,256)
    self.fc5 = nn.Linear(256,512)
    self.fc6 = nn.Linear(512,784)

  def encode(self, x):
    h1 = F.relu(self.fc1(x))
    h1 = F.relu(self.fc2(h1))
    return self.fc31(h1), self.fc32(h1) #mu, logvar

  def reparameterize(self, mu, logvar):
    std = torch.exp(0.5*logvar)
    eps = torch.rand_like(std)
    return mu + eps * std

  def decode(self, z):
    h2 = F.relu(self.fc4(z))
    h2 = F.relu(self.fc5(h2))
    return torch.sigmoid(self.fc6(h2))

  def forward(self, x):
    mu, logvar = self.encode(x.view(-1,784)) # sampling
    z = self.reparameterize(mu,logvar) # reparametrize trick을 이용하여 미분가능하게 변경
    return self.decode(z), mu, logvar

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VAE().to(device)

In [60]:
model

VAE(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc31): Linear(in_features=256, out_features=2, bias=True)
  (fc32): Linear(in_features=256, out_features=2, bias=True)
  (fc4): Linear(in_features=2, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=784, bias=True)
)

In [0]:
optimizer = optim.Adam(model.parameters())

def lossfunction(recon_x, x, mu, logvar):
  BCE = F.binary_cross_entropy(recon_x, x.view(-1,784), reduction='sum')
  # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
  KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
  return BCE + KLD

In [0]:
def train(epoch):
  model.train()
  train_loss = 0
  for batch_idx, (data, _) in enumerate(trainloader):
    data = data.to(device)
    optimizer.zero_grad()

    recon_batch, mu, logvar = model(data)
    loss = lossfunction(recon_batch, data ,mu, logvar)

    loss.backward()
    train_loss += loss.item()
    optimizer.step()

    if batch_idx % 100 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(trainloader.dataset),
          100. * batch_idx / len(trainloader), loss.item() / len(data)))
  print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [0]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(trainloader):
        data = data.to(device)
        optimizer.zero_grad()
        
        recon_batch, mu, logvar = model(data)
        loss = lossfunction(recon_batch, data, mu, logvar)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(trainloader.dataset)))

In [70]:
for epoch in range(1, 51):
    train(epoch)
    test()
    with torch.no_grad():
            sample = torch.randn(64, 2).to(device)
            sample = model.decode(sample).cpu()
            save_image(sample.view(64, 1, 28, 28),
                       '/gdrive/My Drive/' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 159.859365
Train Epoch: 1 [10000/60000 (17%)]	Loss: 159.353828
Train Epoch: 1 [20000/60000 (33%)]	Loss: 157.094590
Train Epoch: 1 [30000/60000 (50%)]	Loss: 151.715234
Train Epoch: 1 [40000/60000 (67%)]	Loss: 159.360439
Train Epoch: 1 [50000/60000 (83%)]	Loss: 160.751758
====> Epoch: 1 Average loss: 154.5627
====> Test set loss: 151.7051
Train Epoch: 2 [0/60000 (0%)]	Loss: 152.091572
Train Epoch: 2 [10000/60000 (17%)]	Loss: 153.983574
Train Epoch: 2 [20000/60000 (33%)]	Loss: 156.447803
Train Epoch: 2 [30000/60000 (50%)]	Loss: 147.338545
Train Epoch: 2 [40000/60000 (67%)]	Loss: 153.793730
Train Epoch: 2 [50000/60000 (83%)]	Loss: 148.678711
====> Epoch: 2 Average loss: 149.9144
====> Test set loss: 149.1920
Train Epoch: 3 [0/60000 (0%)]	Loss: 146.533281
Train Epoch: 3 [10000/60000 (17%)]	Loss: 137.705342
Train Epoch: 3 [20000/60000 (33%)]	Loss: 146.975723
Train Epoch: 3 [30000/60000 (50%)]	Loss: 149.864473
Train Epoch: 3 [40000/60000 (67%)]	Loss: 142.62